In [ ]:
# packages
import pandas as pd
import os
import psycopg2
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
import joblib
from datetime import datetime, timedelta
import numpy as np
import matplotlib.pyplot as plt



In [ ]:
# cursor for postgress SQL
# requrired for DB connection to access DataSet efficiently
def newCursor():
    try:
        connection = psycopg2.connect(user = os.environ["DB_USER"],
                                      host = os.environ["DB_HOST"],
                                      password = os.environ["DB_PASSWORD"],
                                      port = "5432",
                                      database = os.environ["DB_DATABASE"])

        cursor = connection.cursor()
        return cursor

    except (Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL", error)
        return error


In [ ]:
# helper function to get rows from table given schema (api or swift)
# returns a pandas dataframe
def get_rows_by_table(schemaName, tableName, limit=10):    
    c = newCursor()
    q = f"""
    SELECT *
    FROM {schemaName}.{tableName}
    LIMIT {limit}
    """
    c.execute(q)
    cols = [desc[0] for desc in c.description]
    df = pd.DataFrame(c.fetchall(), columns=cols)
    return df

In [1]:
# to get column names from table given schema
def get_cols_by_table(schemaName, tableName):
    c = newCursor()
    q = f"""
    SELECT *
    FROM {schemaName}.{tableName}
    LIMIT 1
    """
    c.execute(q)
    cols = [desc[0] for desc in c.description]
    return cols

In [3]:
# to get all tables name in api schema
def get_api_table_names_by_schema():
    c = newCursor()
    q = f"""
    SELECT table_name
      FROM information_schema.tables
     WHERE table_schema='api'
       AND table_type='BASE TABLE'
    """
    c.execute(q)
    api_tables = c.fetchall()
    return [t[0] for t in api_tables]

In [1]:
def generate_random(year, cols, cur_df):
        
    cur_df[cols] = cur_df[cols].applymap(add_random)
    cur_df['I2: Number of Employee'] = cur_df['I2: Number of Employee'].apply(add_random_people)
    cur_df['I3: Revenue'] = cur_df['I3: Revenue'].apply(add_random_revenue)
    cur_df['I4: Year'] = year
    
    return cur_df
    
def add_random(n):
    new = n + random.randint(-20, 20)
    if new < 0:
        new = random.randint(0, 20)
    elif new > 100:
        new = random.randint(80, 100)
    return new

def add_random_revenue(n):
    new = n * random.uniform(0.7, 1.3)
    return new

def add_random_people(n):
    new = n + random.randint(-n//5, n//5)
    if new < 0:
        new = random.randint(0, 50)
    return new
    

In [ ]:
# pure random for stock price not related to ESG scores
def pure_random():
    # Generate random data
    num_companies = 10000  # Reduced for demonstration purposes
    company_names = [f"Company {i}" for i in range(1, num_companies + 1)]

    data = {'I1: Company Name': [], 'Date': [], 'StockPrice': []}

    for company in company_names:
        start_date = datetime(2013, 1, 1)
        end_date = datetime(2014, 1, 1)
        date_range = [start_date + timedelta(days=i) for i in range((end_date - start_date).days)]

        base_price = random.uniform(1, 600)
        stock_prices = [base_price + random.uniform(-0.1, 20) + i * 0.001 * random.uniform(-1, 1) for i in range(len(date_range))]

        # Append data to the DataFrame
        data['I1: Company Name'].extend([company] * len(date_range))
        data['Date'].extend(date_range)
        data['StockPrice'].extend(stock_prices)

    # Create a DataFrame
    df_stock = pd.DataFrame(data)

    # Group by company and calculate min, max, and avg stock prices
    stock_data = df_stock.groupby('I1: Company Name').agg({'StockPrice': ['min', 'max', 'mean']})

    # Reset column names for easier access
    stock_data.columns = ['Min', 'Max', 'Mean']

    return stock_data

In [ ]:
def apply_esg(df_stock, prob):
    # Calculate adjusted stock prices based on normalized value
    df_stock['Min'] = np.where(df_stock['Probability'] > prob,df_stock['Min']* (df_stock['normalized_value']+1),df_stock['Min'])
    df_stock['Max'] = np.where(df_stock['Probability'] > prob,df_stock['Max']* (df_stock['normalized_value']+1),df_stock['Max'])
    df_stock['Mean'] = np.where(df_stock['Probability'] > prob,df_stock['Mean']* (df_stock['normalized_value']+1),df_stock['Mean'])

    return df_stock

In [ ]:
def add_stock_price(row):
    
    year, rev, esg = row['I4: Year'], row['I3: Revenue'], row['ESG_Score']
    # normalize data
    norm_rev = (rev - min_rev) / (max_rev - min_rev)
    norm_esg = esg / 100
    
    # generate average based on weights and spy return rate of the year. 
    avg_price = random.uniform(0.8, 1.2) * (rev * 0.01 / min_rev) * norm_rev + random.uniform(0, 0.2) * esg * norm_esg
    avg_price *= (1 + spy_return[year] * 0.01)
    min_price = avg_price * (1 - random.uniform(0.1, 0.5))
    max_price = avg_price * (1 + random.uniform(0.1, 0.5))

    data = [avg_price, min_price, max_price]
    formatted = [round(number, 2) for number in data]

    return formatted
    